In [ ]:
# colab 실행시 필요한 다운로드

#Fast api 
#!pip install fastapi nest-asyncio pyngrok uvicorn
# !pip install aiofiles

#qr 코드
#!pip install qrcode

# yolov4 사용 하려면 4.4.0 버전 이상 OpenCv를 사용해야 함
#!pip install --upgrade opencv-python

In [9]:
# 필요한 라이브러리 당겨오기 
from typing import Optional
# Pydantic을 이용해 파이썬 표준 타입으로 분문을 선언 
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# json, qrcode 라이브러리 가져오기 
import json
import qrcode

# FastAPI  및 웹 관련 라이브러리 가져오기 
from fastapi import FastAPI
from fastapi import Request
from fastapi.staticfiles import StaticFiles #new
from fastapi.templating import Jinja2Templates
from fastapi import HTTPException
from core.config import settings
from apis.general_pages.route_homepage import general_pages_router
from fastapi.responses import HTMLResponse

# 모델 2 필요한 것 
import cv2
import numpy as np
# from google.colab.patches import cv2_imshow # colab에서는 이걸 사용해야함
from glob import glob

# json 데이터 가져오기 
with open('./users/account.json', 'r') as f:
    account = json.load(f)
with open('./users/pet_num.json', 'r') as fp:
    data = json.load(fp)


classes = []
# class의 이름들
with open("./data/custom.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# py 파일 가져오기 

Module 1 : 사진파일 >> 텍스트 >> id 값 가져오기

In [11]:
from module import call_picture

In [4]:
# 카드에서 사용자 ID 인식하기 (OCR 이용)
call_picture('./ocr/pet_card2.jpg')

'PET102'

Module 2 : YOLO함수 사용해서 페트병 개수 확인하기

In [3]:
# Module 2 : YOLO 함수
def yolo(frame, size, score_threshold, nms_threshold):
    # Module 2 모델에 필요한 것 
    import cv2
    import numpy as np

    net = cv2.dnn.readNetFromDarknet("./data/yolov4.cfg", "./data/yolov4_last_3000.weights")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    layer_names = net.getLayerNames()

    # net.getUnconnectedOutLayers()의 리턴값이 [000, 000, 000] 이었음
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    # output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    # 클래스의 갯수만큼 랜덤 RGB 배열을 생성
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    # 이미지의 높이, 너비, 채널 받아오기
    height, width, channels = frame.shape

    # 네트워크에 넣기 위한 전처리
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (size, size), (0, 0, 0), True, crop=False)

    # 전처리된 blob 네트워크에 입력
    net.setInput(blob)

    # 결과 받아오기
    outs = net.forward(output_layers)

    # object_data 객체 선언
    # 각 데이터들을 저장할 리스트 선언
    object_data = {
      'class_ids' : [],
      'confidences' : [],
      'boxes' : [],
      'indexes' : -1
    }

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.1:
                # 탐지된 객체의 너비, 높이 및 중앙 좌표값 찾기
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # 객체의 사각형 테두리 중 좌상단 좌표값 찾기
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                object_data['boxes'].append([x, y, w, h])
                object_data['confidences'].append(float(confidence))
                object_data['class_ids'].append(class_id)

    # 후보 박스(x, y, width, height)와 confidence(상자가 물체일 확률) 출력
    # print(f"boxes: {object_data['boxes']}")
    # print(f"confidences: {object_data['confidences']}")

    # Non Maximum Suppression (겹쳐있는 박스 중 confidence 가 가장 높은 박스를 선택)
    indexes = cv2.dnn.NMSBoxes(object_data['boxes'], object_data['confidences'], score_threshold=score_threshold, nms_threshold=nms_threshold)
    object_data['indexes'] = indexes

    # 후보 박스 중 선택된 박스의 인덱스 출력
    # print(f"indexes: ", end='')
    # for index in indexes:
    #     print(index, end=' ')
    # print("\n\n============================== classes ==============================")

    for i in range(len(object_data['boxes'])):
        if i in indexes:
            x, y, w, h = object_data['boxes'][i]
            class_name = classes[object_data['class_ids'][i]]
            label = f"{class_name} {object_data['confidences'][i]:.2f}"
            color = colors[object_data['class_ids'][i]]

            # 사각형 테두리 그리기 및 텍스트 쓰기
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.rectangle(frame, (x - 1, y), (x + len(class_name) * 13 + 65, y - 25), color, -1)
            cv2.putText(frame, label, (x, y - 8), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 0), 2)
            
            # 탐지된 객체의 정보 출력
            print(f"[{class_name}({i})] conf: {object_data['confidences'][i]} / x: {x} / y: {y} / width: {w} / height: {h}")

    return frame, object_data

In [4]:
# Module 2 : pass or fail 알려주고 확인하기 
# 올바르게 분류 됐으면 "올바르게 분류 됐습니다."
# 올바르게 분류 되지 않았으면 "방법에 맞게 분류해주세요."

def pet_total(image_list):
    p_count = 0
    f_count = 0
    n_count = 0

    for img in image_list:
        image = img
        frame = cv2.imread(image)
        size = 256

        frame, object_data = yolo(frame=frame, size=size, score_threshold=0.4, nms_threshold=0.4)
        
        try:
            flag = object_data['class_ids'][object_data['indexes'][0]] # 이걸 판별해서
            #print(f'페트병을 분리함에 넣어주세요.') 
        except:
            #print(f'이미지 인식 실패.')
            n_count += 1
            continue

        if flag == 0:
            p_count += 1
        else: 
            f_count += 1

    #print(f'총 {p_count + f_count + n_count}의 페트병 중 {p_count}개의 페트병이 통과되었고, {n_count}개의 페트병 이미지는 인식하지 못했습니다.')
    return p_count

In [5]:
# 아이디, 페트병 개수, 리워드 현황 출력 
# json 파일로 변경사항 저장 
def pet_sum(user_id:str, pet_num:int):
  if user_id in list(data.keys()):
    data[user_id]+= pet_num
    with open('pet_num.json', 'w') as fp:
      json.dump(data,fp)
    return data[user_id]
  else:
    new_m_p = {user_id : pet_num}
    data.update(new_m_p)
    with open('pet_num.json', 'w') as fp:
      json.dump(data,fp)
    return data[user_id]

In [6]:
# Module 3 : API로 결과 출력 

templates = Jinja2Templates(directory="templates")

def include_router(app):
    app.include_router(general_pages_router)


def configure_static(app):  #new
    app.mount("/static", StaticFiles(directory="static"), name="static")


def start_application():
    app = FastAPI(title=settings.PROJECT_NAME,version=settings.PROJECT_VERSION)
    include_router(app)
    configure_static(app) #new
    return app

app = start_application()

# 웹페이지만 나옴 
@app.get('/login/')
def login(request: Request):
    return templates.TemplateResponse("auth/login.html", {"request": request})

# 페트병 개수 
@app.get('/pet/', response_class=HTMLResponse)
async def pet(request: Request):
    # 사용자 정보 
    card = 'pet_card.jpg'
    user_id = call_picture(card)
    # 페드병 개수 
    image_list = glob('./image/*/*/*.jpg')
    pet_num = pet_total(image_list)

    # 결과
    result = pet_sum(user_id, pet_num)
    img = qrcode.make(f'{ngrok_tunnel.public_url}/pet/?pet_num={pet_num}')
    img.save('result.png')
    return templates.TemplateResponse("main.html", {"request": request, 'id':user_id, 'pet_num':result})


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [13712]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://c8fb-114-203-149-236.ngrok.io


[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU


[clean(0)] conf: 0.4139142632484436 / x: 774 / y: 403 / width: 1175 / height: 3670


[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU


...

[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU


INFO:     127.0.0.1:62779 - "GET /pet/ HTTP/1.1" 200 OK


[ WARN:0] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU


INFO:     127.0.0.1:62779 - "GET /static/images/nolabel.png HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [13712]


해야하는 일 
1. main 파일 정리하기 (함수들은 module.py파일로 묶어서 가져오기 유도)
- module.py에 ocr 코드는 삽입 완료 
- yolo 함수등은 시도했으나 실패